In [1]:
from __future__ import division
from sklearn.preprocessing import CategoricalEncoder
from sklearn.preprocessing import LabelEncoder

import torchvision.models as models
from multiprocessing import Pool
import time
import numpy as np
import pandas as pd
import datetime as dt
import os.path as dpath
import matplotlib.pyplot as plt
from os import listdir, makedirs, getcwd, remove
from os.path import isfile, join, abspath, exists, isdir, expanduser
from sklearn.model_selection import train_test_split
from mpl_toolkits.axes_grid1 import ImageGrid
from PIL import Image
import torch
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms, datasets, models
import pdb
from pathlib import Path
import torch.optim as optim


import torch.nn as nn
import torch.nn.functional as F

from IPython.display import FileLink, FileLinks


%matplotlib inline

from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [2]:
path = "/home/edwin/Datasets/competitions/avito-demand-prediction/"
PATH = Path(path)

In [3]:
TRN_CSV = pd.read_csv(PATH/'train.csv')
TRN_CSV_COPY = TRN_CSV.copy()

TEST_CSV = pd.read_csv(PATH/'test.csv')
TEST_CSV_COPY = TEST_CSV.copy()

In [4]:
n = len(TRN_CSV_COPY); n

1503424

In [5]:
n_test = len(TEST_CSV_COPY); n_test

508438

In [6]:
idxs = get_cv_idxs(n, val_pct=15000/n)

In [8]:
# ?? get_cv_idxs

In [9]:
len(idxs)

15000

In [12]:
class AvitoDataset(Dataset):
    def __init__(self, df):
        self.df = df
        self.n = len(df)
        self.cont_vars = ['price']
        self.cat_vars = ['region']
        self.y_var = 'deal_probability'
        
        self.df_cont = df[self.cont_vars].fillna(0)
        self.df_cat = df[self.cat_vars]
        
        for c in self.cont_vars:
            self.df_cont[c] = self.df_cont[c].astype("float32")
        
        for c in self.cat_vars:
            self.df_cat[c] = self.df_cat[c].astype('category').cat.as_ordered()
            
        for n, c in self.df_cat.items():
            self.df_cat[n] = self.df_cat[n].cat.codes+1
            
        self.conts = np.stack([c.values for n, c in self.df_cont.items()], 1).astype(np.float32)
        self.cats = np.stack([c.values for n, c in self.df_cat.items()], 1).astype(np.int64)
        
        if self.y_var in df:
            self.y = df['deal_probability'][:, None].astype(np.float32)
        else:
            self.y = np.zeros([self.n,1]).astype(np.float32)
    
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        x_cat = self.cats[idx]
        x_cont = self.conts[idx]
        y = self.y[idx]
        return [x_cat, x_cont, y]
    

In [13]:
n = len(TRN_CSV_COPY)

In [14]:
train_ratio = 0.75

In [15]:
train_size = int(n * train_ratio); train_size

1127568

In [16]:
val_idxs = list(range(train_size, n))

In [17]:
len(TRN_CSV_COPY.iloc[val_idxs])

375856

In [18]:
train_ds = AvitoDataset(TRN_CSV_COPY); train_ds[0]

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[array([20]), array([ 400.], dtype=float32), array([ 0.12789], dtype=float32)]

In [19]:
valid_ds = AvitoDataset(TRN_CSV_COPY.iloc[val_idxs])

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [20]:
test_ds = AvitoDataset(TEST_CSV_COPY)

/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/home/edwin/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [22]:
train_dl = DataLoader(train_ds, batch_size=64, num_workers=1, shuffle=False)
valid_dl = DataLoader(valid_ds, batch_size=64, num_workers=1, shuffle=False)
test_dl = DataLoader(test_ds, batch_size=64, num_workers=1, shuffle=False)

In [32]:
class AvitoMixedModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embs = nn.ModuleList([nn.Embedding(29, 14)])
        self.bn = nn.BatchNorm1d(1)
        self.fc1 = nn.Linear(15, 100)
        self.fc2 = nn.Linear(100, 1)
        self.emb_drop = nn.Dropout(0.04)
        
    def forward(self, x_cat, x_cont):
        x = [e(v_cat[:,i]) for i, e in enumerate(self.embs)]
        x = torch.cat(x, 1)
        x = self.emb_drop(x)
        x2 = self.bn(x_cont)
        x = torch.cat([x, x2], 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = x*(1-0)
        x = x+0
        return x
        

In [33]:
model = AvitoMixedModel()

In [34]:
model.cuda()

AvitoMixedModel(
  (embs): ModuleList(
    (0): Embedding(29, 14)
  )
  (bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=15, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
)

In [35]:
lr = 1e-3

In [36]:
optimizer = torch.optim.Adam(model.parameters(), lr=lr); optimizer

In [37]:
criterion = nn.MSELoss()

In [42]:
num_epochs = 15

In [ ]:
# RMSE Error: Need to calculate 1/n sum(y - y_hat)^2 

In [ ]:
# description
# image quality

In [43]:
for epoch in range(num_epochs):    
    for i, data in enumerate(train_dl,0):
        x_cat, x_cont, y = data
        v_cat = Variable(x_cat.cuda()); v_cont = Variable(x_cont.cuda()); vy = Variable(y.cuda());

        optimizer.zero_grad()

        outputs = model(v_cat, v_cont)
        loss = criterion(outputs, vy)
        loss.backward()
        optimizer.step()
        if i % 5000 == 0:
            total = 0
            correct = 0
            for data in valid_dl:
                x_cat, x_cont, y = data
                v_cat = Variable(x_cat.cuda()); v_cont = Variable(x_cont.cuda()); vy = Variable(y.cuda());
                outputs = model(v_cat, v_cont)
                total += vy.size(0)
                correct += ((outputs >= (vy-0.05*vy)) & (outputs <= (vy+0.05*vy))).data.cpu().numpy().sum()
            accuracy = 100 * correct / total
            print("epoch: {}, i:{}, loss:{}, valid accuracy:{} {}/{}".format(epoch, i,loss.data[0], 
                                                                             accuracy, correct / total))    
print("Finished Training")

epoch: 0, i:0, loss:0.08286580443382263, accuracy:1.2837363245498277
epoch: 0, i:5000, loss:0.07823343575000763, accuracy:1.2728278915329274
epoch: 0, i:10000, loss:0.07147502899169922, accuracy:1.2063130560640245
epoch: 0, i:15000, loss:0.056447580456733704, accuracy:1.1999276318590097
epoch: 0, i:20000, loss:0.05895040184259415, accuracy:1.2635158145672811
epoch: 1, i:0, loss:0.08273249119520187, accuracy:1.2619194585160274
epoch: 1, i:5000, loss:0.07833738625049591, accuracy:1.3723340853944064
epoch: 1, i:10000, loss:0.0709594190120697, accuracy:1.191679792260866
epoch: 1, i:15000, loss:0.05659712105989456, accuracy:1.213496658294666
epoch: 1, i:20000, loss:0.058524806052446365, accuracy:1.2260014473628198
epoch: 2, i:0, loss:0.08266451954841614, accuracy:1.3135349708398962
epoch: 2, i:5000, loss:0.07790608704090118, accuracy:1.3172598016261547
epoch: 2, i:10000, loss:0.07058590650558472, accuracy:1.1491102975607679
epoch: 2, i:15000, loss:0.056659404188394547, accuracy:1.4313992592

In [58]:
model

AvitoMixedModel(
  (embs): ModuleList(
    (0): Embedding(29, 14)
  )
  (bn): BatchNorm1d(1, eps=1e-05, momentum=0.1, affine=True)
  (fc1): Linear(in_features=15, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=1, bias=True)
  (emb_drop): Dropout(p=0.04)
)

In [60]:
torch.save(model.state_dict(), 'weights_0520.pkl')

In [67]:
res = []
for i, data in enumerate(test_dl, 0):
    x_cat, x_cont, y = data
    v_cat = Variable(x_cat.cuda()); v_cont = Variable(x_cont.cuda()); vy = Variable(y.cuda());
    outputs = model(v_cat, v_cont)
    res.append(outputs.data.cpu().numpy())
predictions = np.concatenate(res).ravel()

In [69]:
NEW_SUBMIT_CSV = pd.DataFrame()
NEW_SUBMIT_CSV['item_id'] = TEST_CSV_COPY.item_id.values
NEW_SUBMIT_CSV['deal_probability'] = predictions
NEW_SUBMIT_CSV.to_csv('new_submission.csv', index=False)
FileLink('new_submission.csv')

/home/edwin/Developer/iMaterial/avito-demand-prediction/new_submission.csv